In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
# Analyses are similar to FRN, but with different electrodes
# First, we get the 800 timepoints we are considering as a list, to use for indices given by cluster-based permutation later
# But only grab -0.25 to 1.5 sec time-locked to feedback onset (idx= 350:701)
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')
pp = 0 #only need one participant

# we can use aligned data

id_directory = os.path.join(data_directory, 'p%03d/' % pp)
pp_directory = os.path.join(id_directory, erps)
fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s_%s-ave.fif' % (pp, 'EarlyLate', 'P3', 'aligned', 'all'))
evoked = mne.read_evokeds(fname)
df = evoked[0].to_data_frame()
time = df['time'].tolist()
time = time[350:701] #get only timepoints we want

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_EarlyLate_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)


In [3]:
# Load data for each condition
# Take the mean across electrodes of interest per participant
# transform data into array accepted for cluster-based permutation test in mne
# output: (n_participants, n_timepts) for each condition

root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#specify channels we need - P3
channels = ['FCz', 'Fz', 'Cz', 'Pz']

    
#read in evoked object
tasknames = ['rot', 'rdm', 'mir']
conditionnames = [0, 1]


#aligned
evoked_list = []

for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s_%s-ave.fif' % (participants[pp], 'EarlyLate', 'P3', 'aligned', 'all'))
    evoked = mne.read_evokeds(fname)
    evoked = evoked[0]
    evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
    evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
    evoked = evoked[350:701] #get timepoints we need
    evoked_list.append(evoked)
    
aligned_flist = evoked_list

#rot and rdm and mir
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s_%s-ave.fif' % (participants[pp], 'EarlyLate', 'P3', tasknames[task], conditionnames[condition]))
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
            evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
            evoked = evoked[350:701] #get timepoints we need
            evoked_list.append(evoked)
        if task == 0 and condition == 0:
            early_rot_flist = evoked_list
        elif task == 0 and condition == 1:
            late_rot_flist = evoked_list
        elif task == 1 and condition == 0:
            early_rdm_flist = evoked_list
        elif task == 1 and condition == 1:
            late_rdm_flist = evoked_list
        elif task == 2 and condition == 0:
            early_mir_flist = evoked_list
        elif task == 2 and condition == 1:
            late_mir_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_EarlyLate_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_EarlyLate_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_EarlyLate_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.

        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_EarlyLate_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_EarlyLate_P3_aligned_all-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\frn\p007_EarlyLate_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_EarlyLate_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\frn\p009_EarlyLate_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\frn\p027_EarlyLate_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\frn\p028_EarlyLate_P3_rot_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\frn\p029_EarlyLate_P3_rot_0-ave.fif ...


        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\frn\p015_EarlyLate_P3_rot_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\frn\p016_EarlyLate_P3_rot_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (basel

...

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p010\frn\p010_EarlyLate_P3_rdm_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p011\frn\p011_EarlyLate_P3_rdm_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p012\frn\p012_EarlyLate_P3_rdm_1-ave.fif ...


        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\frn\p030_EarlyLate_P3_rdm_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\frn\p031_EarlyLate_P3_rdm_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (basel

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p018\frn\p018_EarlyLate_P3_mir_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p019\frn\p019_EarlyLate_P3_mir_0-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No 

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p005\frn\p005_EarlyLate_P3_mir_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\frn\p006_EarlyLate_P3_mir_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\frn\p007_EarlyLate_P3_mir_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p025\frn\p025_EarlyLate_P3_mir_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\frn\p026_EarlyLate_P3_mir_1-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 44 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\frn\p027_EarlyLate_P3_mir_1-ave.fif ...


In [4]:
# Comparing two ERP signals is just the same as taking their difference (ERP1 minus ERP2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
#     np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [5]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in ERP plots

flists = [early_rot_flist, late_rot_flist, early_rdm_flist, late_rdm_flist, early_mir_flist, late_mir_flist]
conditionnames = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], aligned_flist, p, perms)
#     print(clust_idx, clust_pvals)
    if len(clust_idx) == 0:
        condition.append(conditionnames[c])
        clust_idx_start.append(np.nan)
        clust_idx_end.append(np.nan)
        time_start.append(np.nan)
        time_end.append(np.nan)
        p_values.append(np.nan)
    else:
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_vsAligned_P3.csv')
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-2.847671 max=3.053348
Running initial clustering …
Found 4 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.044948 max=2.868729
Running initial clustering …
Found 6 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.291559 max=3.808079
Running initial clustering …
Found 5 clusters


...

  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.141099 max=3.357262
Running initial clustering …
Found 2 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.505222 max=2.475379
Running initial clustering …
Found 4 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

In [6]:
# Next, we subtract aligned from each condition, so that we can compare early vs late in each perturbation type
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
earlyrot_diff = []
laterot_diff = []
earlyrdm_diff = []
laterdm_diff = []
earlymir_diff = []
latemir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(early_rot_flist, aligned_flist)
        earlyrot_diff.append(diffevks)
        earlyrot_diff = earlyrot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(late_rot_flist, aligned_flist)
        laterot_diff.append(diffevks)
        laterot_diff = laterot_diff[0]
    elif cond == 2:
        diffevks = np.subtract(early_rdm_flist, aligned_flist)
        earlyrdm_diff.append(diffevks)
        earlyrdm_diff = earlyrdm_diff[0]
    elif cond == 3:
        diffevks = np.subtract(late_rdm_flist, aligned_flist)
        laterdm_diff.append(diffevks)
        laterdm_diff = laterdm_diff[0]
    elif cond == 4:
        diffevks = np.subtract(early_mir_flist, aligned_flist)
        earlymir_diff.append(diffevks)
        earlymir_diff = earlymir_diff[0]
    elif cond == 5:
        diffevks = np.subtract(late_mir_flist, aligned_flist)
        latemir_diff.append(diffevks)
        latemir_diff =latemir_diff[0]

In [7]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of EARLY VS LATE comparisons
# This information can then be included in ERP difference wave plots

conds = ['rot_diff', 'rdm_diff', 'mir_diff']
conditionnames = ['rot', 'rdm', 'mir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(laterot_diff, earlyrot_diff, p, perms)
#         print(clust_idx, clust_pvals)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 1:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(laterdm_diff, earlyrdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:    
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 2:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(latemir_diff, earlymir_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:    
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_EarlyvsLate_P3.csv')
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-0.891236 max=3.701199
Running initial clustering …
Found 6 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.922236 max=1.385855
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-1.564736 max=0.184312
Running initial clustering …
Found 0 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_19280\4149227759.py:10: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_19280\4149227759.py:10: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


In [8]:
# Next step is to subtract early from late condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
rot_diff = []
rdm_diff = []
mir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(laterot_diff, earlyrot_diff)
        rot_diff.append(diffevks)
        rot_diff = rot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(laterdm_diff, earlyrdm_diff)
        rdm_diff.append(diffevks)
        rdm_diff = rdm_diff[0]
    elif cond == 2:
        diffevks = np.subtract(latemir_diff, earlymir_diff)
        mir_diff.append(diffevks)
        mir_diff = mir_diff[0]

In [9]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of PERTURBATION comparisons
# This information can then be included in ERP difference wave plots

conds = ['rotvmir', 'rotvrdm', 'mirvrdm']
conditionnames = ['rotvmir', 'rotvrdm', 'mirvrdm']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, mir_diff, p, perms)
#         print(clust_idx, clust_pvals)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 1:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, rdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 2:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(mir_diff, rdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_PerturbTypeComp_P3.csv')
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=0.346364 max=2.573674
Running initial clustering …
Found 6 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-0.158017 max=3.414517
Running initial clustering …
Found 3 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.337075 max=1.028262
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]